# Actividad 11 2018-1, I/O: archivos y *bytes*

Ocuparemos [esta actividad](https://github.com/IIC2233/Syllabus-2018-1/blob/master/Actividades/AC11/AC11.pdf) de hace algunos semestres para guiarlos en el proceso de leer y entender un enunciado, decidir dónde aplicar la materia y, finalmente, implementar. Recuerden el orden en que se hace este proceso: leer, entender, aplicar la materia y programar.

## Leyendo el enunciado

Primero, obviamente, partiremos por el proceso de leer el enunciado. Sigan el link, lean la actividad por completo y, una vez que terminen de revisar todo (incluidas las Notas), vuelvan a este archivo.

## Parte 1: Encontrar archivos

1. Ejecutar el archivo `esconder.py`, lo podemos hacer desde nuestro editor o de cualquier otra parte.
2. Se crearán las carpetas, lo que generará algo como lo de la imagen:

![](img/folders.jpg)

3. En alguna de las carpetas se esconderán los archivos corruptos (`marciano64.png` y `marcianozurdo.pep`).
4. Encontrar automáticamente la ruta a cada archivo.

En esta parte, estamos trabajando con archivos y rutas, por lo que debemos recurrir a la materia de *paths*. Lo que queremos hacer es entrar a cada una de esas carpetas, que dentro tendrán otras carpetas, hasta encontrar los archivos buscados: 

In [1]:
import os

Nuestra idea se ve algo así:

0. A partir del directorio actual (".")
1. Mira los elementos dentro de cada carpeta
    1. Si el elemento es un archivo, compara si es el buscado
    2. Si el elemento es una carpeta, mira sus contenidos (repetir 1, 1A y 1B)

In [2]:
def mirar_contenidos(carpeta):
    # `os.listdir` solo nos da el nombre...
    for elemento in os.listdir(carpeta):
        # ... asi que para tener la ruta ocupamos `os.path.join`
        ruta = os.path.join(carpeta, elemento)
        if os.path.isfile(ruta):
            # Ahora que sabemos que es un archivo, podemos extraer su nombre
            nombre = os.path.basename(ruta)
            if nombre in ["marciano64.png", "marcianozurdo.pep"]:
                # Si su nombre está entre los buscados, imprimimos la ruta
                print(ruta)
        elif os.path.isdir(ruta):
            mirar_contenidos(ruta)

mirar_contenidos(".")

./potato3/potato3/marciano64.png
./potato3/potato3/marcianozurdo.pep


Para esta parte, también podemos ocupar el método `os.walk(raiz)`, que nos permite recorrer las carpetas y archivos a partir de una raíz. En este caso, podemos partir desde el directorio actual (ya que las carpetas estarán junto a este archivo .py) y mirar dentro de las carpetas creadas si es que están los archivos buscados.

In [3]:
# Subcarpeta es una subcarpeta dentro del directorio actual (".")
# Carpetas son las carpetas dentro de "subcarpeta"
# Archivos son los archivos dentro de "subcarpeta"
for subcarpeta, carpetas, archivos in os.walk("."):
    # Para este problema solo nos interesa revisar los archivos
    for archivo in archivos:
        # Si el archivo se llama "marciano64.png" o "marcianozurdo.pep"
        # es el archivo que estamos buscando
        if archivo in ["marciano64.png", "marcianozurdo.pep"]:
            # Imprimimos la ruta relativa al archivo
            print(os.path.join(subcarpeta, archivo))


./potato3/potato3/marciano64.png
./potato3/potato3/marcianozurdo.pep


Ahora, para facilitar nuestro uso posterior, crearemos un método que busque un archivo específico:

In [4]:
def buscar_archivo(archivo_buscado):
    for subcarpeta, carpetas, archivos in os.walk("."):
        for archivo in archivos:
            if archivo == archivo_buscado:
                return os.path.join(subcarpeta, archivo)

archivo_1 = buscar_archivo("marcianozurdo.pep")
archivo_2 = buscar_archivo("marciano64.png")


**Importante:** Como podrán ver, esta parte se puede hacer de muchas formas, ya sea ocupando métodos que conozcan (que están en los contenidos) o métodos que no conozcan tanto pero que encuentren buscando en internet. Por ejemplo, si buscamos `python iterate folder` (tanto en Google como en DuckDuckGo) el primer resultado es [esta pregunta en StackOverflow](https://stackoverflow.com/questions/10377998/how-can-i-iterate-over-files-in-a-given-directory) donde hay muchas respuestas posibles, entre ellas la opción de utilizar `os.listdir` y un poco más abajo la de utilizar `os.walk`, entre otras que no se muestran aquí. Lo importante es que busquen, encuentren y prueben. Usar internet durante una actividad está permitido.

## Parte 2: Algoritmos

### 1. Algoritmo base64

El enunciado nos indica los pasos a seguir:

In [5]:
import string

# Ocuparemos este diccionario para asignar un número a cada caracter
caracteres = string.ascii_uppercase + \
            string.ascii_lowercase + \
            string.digits + \
            "+" + "/"
base_to_num = dict(zip(caracteres, range(0,64)))
# Este código es equivalente a hacer:
# 
# base_to_num = dict()
# i = 0
# for caracter in caracteres:
#     base_to_num[caracter] = i
#     i += 1


def algoritmo_base_64(archivo):
    # Leemos los bytes del archivo
    with open(archivo, "rb") as f:
        conjunto_bits = ""
        for byte in f.read():
            # Transformamos cada byte a ASCII
            byte_como_ascii = str(chr(byte))
            # Cada caracter lo cambiamos a base64
            caracter_base64 = base_to_num[byte_como_ascii]
            # Convertimos cada valor a su versión binaria de 6 bits
            caracter_bin6 = bin(caracter_base64)[2:].zfill(6)
            # Concatenamos los conjuntos de 6 bits
            conjunto_bits += caracter_bin6

    # Ahora agrupamos de a 8 bytes
    # Y cada grupo se convierte a decimal
    output = bytearray() # Ocuparemos bytearray pero funciona como lista
    for indice in range(0, len(conjunto_bits), 8):
        # Grupo de 8 bytes
        grupo = conjunto_bits[indice:indice+8]
        # Convertir a decimal
        grupo_como_int = int(grupo, 2)
        output.append(grupo_como_int)
    
    return output


### 2. Algoritmo Rotar hacia la izquierda (*rotate left*)

Es un algoritmo simple donde el primer elemento pasa al final, y los demás se corren

In [6]:
# Como no sabemos sobre qué se aplicará el algoritmo asumiremos una lista 
def rotate_left(chunk):
    primero = chunk.pop(0)
    return chunk + [primero]


## Parte 3: Juntar los archivos

Nos dicen que el archivo resultante debería llamarse `resultado.png`, y que debería visualizarse correctamente como imagen.

In [7]:
ruta_resultado = "resultado.png"

Esta sección es bastante larga y se ve complicada a primera vista. Primero escribiremos cómo haríamos esta función en seudocódigo, sin programar en Python (aún), y dejaremos a mano la imagen de la explicación del algoritmo:

![](img/diagram.jpg)

1. Debemos tener ambos archivos como un arreglo de bytes, los que iremos recorriendo al mismo tiempo.
2. Necesitamos saber el tamaño de cada *chunk*, por lo que necesitamos ir avanzando en los dígitos de Fibonacci de forma ordenada. Podemos hacer esto con una función, con un generador o simplemente manejando las variables de forma inteligente (lo que se hace en esta solución).
3. Aplicamos la transformación correspondiente en cada *chunk* impar (el segundo algoritmo, *rotate left*).
4. Vamos actualizando el resultado.
5. Guardamos el resultado con el nombre "resultado.png".

Si les interesa conocer la versión de Fibonacci con generadores, se la dejamos a continuación (esta materia fue vista en el taller de programación funcional)

In [8]:
def fibonacci():
    # Recordemos que en una secuencia de fibonacci, se parte con 1 y 1
    # y el siguiente número es la suma de los dos anteriores:
    # 1, 1, 2, 3, 5, 8, ...
    a, b = 1, 1
    while True:
        yield a
        a, b = b, a + b


Ahora, cargaremos los bytes de los archivos:

In [9]:
# Guardamos los bytes del primer archivo
# marcianozurdo.pep
with open(archivo_1, "rb") as f:
    bytes_archivo_1 = f.read()

# Guardamos los bytes del segundo archivo
# con el algoritmo ya aplicado
# marciano64.png
bytes_archivo_2 = algoritmo_base_64(archivo_2)


Finalmente, implementamos la unión de los *chunks*:

In [10]:
# Almacenaremos el resultado en un bytearray
resultado = bytearray()
# (En la versión con la función generadora, instanciamos el generador)
# fib = fibonacci()

# Dígitos iniciales de Fibonacci
a, b = 1, 1

# Mientras tenga bytes para agregar, hago el siguiente proceso
while (bytes_archivo_1 + bytes_archivo_2):
    # Inicialmente, los chunks serán de tamaño 1 y 1, por lo que
    # no cambian los valores de a y b, y se hace al final de la iteración

    # (La versión con generadores llama next para obtener el siguiente dígito)
    # a, b = next(fib), next(fib)
    

    # Hacemos esta transformación porque en nuestra función asumimos una lista
    chunk1 = bytes_archivo_1[0:a]
    chunk1 = bytearray(rotate_left(list(chunk1)))
    chunk2 = bytes_archivo_2[0:b]
    
    # Unimos los chunks a nuestro resultado
    resultado.extend(chunk1)
    resultado.extend(chunk2)
    
    # Quitamos los bytes ya utilizados
    bytes_archivo_1 = bytes_archivo_1[a:]
    bytes_archivo_2 = bytes_archivo_2[b:]
    
    # Ahora, avanzamos los dígitos de fibonacci
    # (En la versión con generadores no hacemos esto, la función lo hace)
    a = a + b # El siguiente digito es la suma de los dos ultimos
    b = a + b # Ahora el ultimo es el "nuevo a", asi que lo sumamos con b


Finalmente, creamos el archivo con el resultado:

In [11]:
with open(ruta_resultado, "wb") as file:
    file.write(resultado)


Si todo funciona bien, en la siguiente celda podrás ver la imagen del resultado que generamos:

![](resultado.png)